In [13]:
# ===== YOLOv2 (Darknet) | saída em arquivo =====
import os, subprocess
from PIL import Image

IMG = "/content/test2.jpeg"
OUT_TXT = "/content/yolov2_output.txt"
OUT_IMG = "/content/yolov2_pred.jpg"

# Garantir imagem
if not os.path.exists(IMG):
    !wget -O /content/test.jpg https://raw.githubusercontent.com/pjreddie/darknet/master/data/dog.jpg

# Darknet
if not os.path.exists("/content/darknet/darknet"):
    %cd /content
    !git clone https://github.com/AlexeyAB/darknet
    %cd darknet
    !make -j2
else:
    %cd /content/darknet

# Pesos / cfg
!wget -q -nc https://pjreddie.com/media/files/yolov2.weights
!wget -q -nc https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov2.cfg
!wget -q -nc https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/coco.data
!wget -q -nc https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names

# Inferência (captura stdout)
result = subprocess.check_output(
    f"./darknet detector test cfg/coco.data yolov2.cfg yolov2.weights -dont_show -ext_output {IMG}",
    shell=True,
    text=True
)

# Salvar saída textual
with open(OUT_TXT, "w") as f:
    f.write(result)

# Salvar imagem
Image.open("predictions.jpg").save(OUT_IMG)

print("YOLOv2 finalizado")
print("Arquivo:", OUT_TXT)
print("Imagem :", OUT_IMG)


/content/darknet
YOLOv2 finalizado
Arquivo: /content/yolov2_output.txt
Imagem : /content/yolov2_pred.jpg


In [14]:
# ===== YOLOv3 (Darknet) | saída em arquivo =====
import subprocess
from PIL import Image

IMG = "/content/test2.jpeg"
OUT_TXT = "/content/yolov3_output.txt"
OUT_IMG = "/content/yolov3_pred.jpg"

%cd /content/darknet

# Pesos / cfg
!wget -q -nc https://pjreddie.com/media/files/yolov3.weights
!wget -q -nc https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

# Inferência
result = subprocess.check_output(
    f"./darknet detector test cfg/coco.data yolov3.cfg yolov3.weights -dont_show -ext_output {IMG}",
    shell=True,
    text=True
)

# Salvar saída textual
with open(OUT_TXT, "w") as f:
    f.write(result)

# Salvar imagem
Image.open("predictions.jpg").save(OUT_IMG)

print("YOLOv3 finalizado")
print("Arquivo:", OUT_TXT)
print("Imagem :", OUT_IMG)


/content/darknet
YOLOv3 finalizado
Arquivo: /content/yolov3_output.txt
Imagem : /content/yolov3_pred.jpg


In [16]:
# ===== YOLOv8 (Ultralytics) | saída em JSON =====
!pip -q install ultralytics

import json, time
from ultralytics import YOLO
from PIL import Image

IMG = "/content/test2.jpeg"
OUT_JSON = "/content/yolov8_output.json"
OUT_IMG = "/content/yolov8_pred.jpg"

model = YOLO("yolov8n.pt")

t0 = time.time()
results = model(IMG, conf=0.25, verbose=False)
dt = time.time() - t0

boxes = results[0].boxes
names = model.names

data = {
    "model": "yolov8n",
    "image": IMG,
    "inference_time_sec": dt,
    "num_detections": int(len(boxes)),
    "detections": []
}

for i in range(len(boxes)):
    cls = int(boxes.cls[i])
    data["detections"].append({
        "id": i,
        "class_id": cls,
        "class_name": names[cls],
        "confidence": float(boxes.conf[i]),
        "bbox_xyxy": boxes.xyxy[i].tolist()
    })

# Salvar JSON
with open(OUT_JSON, "w") as f:
    json.dump(data, f, indent=2)

# Salvar imagem
results[0].save(filename=OUT_IMG)

print("YOLOv8 finalizado")
print("Arquivo:", OUT_JSON)
print("Imagem :", OUT_IMG)


YOLOv8 finalizado
Arquivo: /content/yolov8_output.json
Imagem : /content/yolov8_pred.jpg
